In [47]:
from src import utils
from copy import deepcopy
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd



In [48]:
 
# Buat path menuju data train, test, dan valid
X_train_data_path = './data/interim/X_train.pkl'
y_train_data_path = './data/interim/y_train.pkl' 
X_test_data_path  = './data/interim/X_test.pkl' 
y_test_data_path  = './data/interim/y_test.pkl' 
X_valid_data_path = './data/interim/X_valid.pkl' 
y_valid_data_path = './data/interim/y_valid.pkl'


# Panggil fungsi deserialize_data dari utils
X_train = utils.deserialize_data(X_train_data_path)
y_train = utils.deserialize_data(y_train_data_path)
X_test  = utils.deserialize_data(X_test_data_path) 
y_test  = utils.deserialize_data(y_test_data_path)
X_valid = utils.deserialize_data(X_valid_data_path)
y_valid = utils.deserialize_data(y_valid_data_path)

# # Verifikasi bahwa data telah dimuat dengan benar
print(f'X_train shape: {X_train.shape}') 
print(f'y_train shape: {y_train.shape}')
print(f'X_test  shape: {X_test.shape}')
print(f'y_test  shape: {y_test.shape}')
print(f'X_valid shape: {X_valid.shape}')
print(f'y_valid shape: {y_valid.shape}')

X_train shape: (26064, 11)
y_train shape: (26064,)
X_test  shape: (3259, 11)
y_test  shape: (3259,)
X_valid shape: (3258, 11)
y_valid shape: (3258,)


### Dropping Duplicates

In [49]:
def drop_duplicate_data(X, y):
    """
    Fungsi ini bertujuan untuk menghapus data duplikat dari dataset.
    
    Parameters:
    X (pd.DataFrame): Dataframe yang berisi data fitur.
    y (pd.Series)   : Series yang berisi data target.

    Returns:
    X (pd.DataFrame): Dataframe setelah data duplikat dihapus.
    y (pd.Series)   : Series setelah data duplikat dihapus.
    
    """
    
    # Validasi parameter
    if not isinstance(X, pd.DataFrame):
        raise ValueError("Parameter X harus bertipe dataframe")
    if not isinstance(y, pd.Series):
        raise ValueError("Parameter y harus bertipe series")
    print("Fungsi drop_duplicate_data: parameter telah divalidasi.")
    
    # Copy data untuk menjaga keaslian data asli
    X = X.copy()
    y = y.copy()
    
    print(f"Fungsi drop_duplicate_data: shape dataset sebelum dropping duplicate adalah {X.shape}.")
    
    # Pengecekan duplicate di X
    X_duplicate = X[X.duplicated()]
    print(f"Fungsi drop_duplicate_data: shape dari data yang duplicate adalah {X_duplicate.shape}.")
    
    # Kalkulasi shape setelah drop duplicate
    X_clean = (X.shape[0] - X_duplicate.shape[0], X.shape[1])
    
    # Print prediksi shape setelah drop duplicate
    print(f"Fungsi drop_duplicate_data: shape dataset setelah drop duplicate seharusnya adalah {X_clean}.")
    
    # Hapus duplicate
    X.drop_duplicates(inplace = True)
    
    # Pengecekan duplicate di y
    y = y.loc[X.index]
    print(f"Fungsi drop_duplicate_data: shape dataset setelah dropping duplicate adalah {X.shape}.")
    
    return X, y


In [50]:
X_train, y_train = drop_duplicate_data(X = X_train, y = y_train)

Fungsi drop_duplicate_data: parameter telah divalidasi.
Fungsi drop_duplicate_data: shape dataset sebelum dropping duplicate adalah (26064, 11).
Fungsi drop_duplicate_data: shape dari data yang duplicate adalah (96, 11).
Fungsi drop_duplicate_data: shape dataset setelah drop duplicate seharusnya adalah (25968, 11).
Fungsi drop_duplicate_data: shape dataset setelah dropping duplicate adalah (25968, 11).


### Imputing null value

In [51]:
def median_imputation(data, subset_data, fit):
    """
    Fungsi ini digunakan untuk imputasi data numerik dengan nilai median.

    Parameters:
    data        (pd.DataFrame)  : Dataframe yang berisi data fitur (train, test, atau valid) yang ingin diimputasi datanya.
    subset_data (list atau dict): Jika fit = True, list nama kolom yang ingin diimputasi. Jika fit = False, dict dengan key 
                                  sebagai nama kolom dan value sebagai nilai median kolom tersebut.
    fit         (bool)          : Jika True, melakukan kalkulasi median. Jika False, melakukan imputasi berdasarkan median 
                                  yang telah dikalkulasi sebelumnya.

    Returns:
    dict atau pd.DataFrame      : Jika fit = True, mengembalikan dict dengan median tiap kolom. Jika fit = False, kembalikan 
                                  DataFrame setelah imputasi.
    """
    
    # Validasi parameter
    if not isinstance(data, pd.DataFrame):
        raise RuntimeError("Fungsi median_imputation: parameter data haruslah bertipe DataFrame!")

    if fit == True:
        if not isinstance(subset_data, list):
            raise RuntimeError("Fungsi median_imputation: untuk nilai parameter fit = True, subset_data harus bertipe list dan " 
                               "berisi daftar nama kolom yang ingin dicari nilai mediannya guna menjadi data imputasi pada kolom "
                               "kolom tersebut.")
    elif fit == False:
        if not isinstance(subset_data, dict):
            raise RuntimeError("Fungsi median_imputation: untuk nilai parameter fit = False, subset_data harus bertipe dict " 
                               "dan berisi key yang merupakan nama kolom beserta value yang merupakan nilai median dari kolom "
                               "tersebut.")
    else:
        raise RuntimeError("Fungsi median_imputation: parameter fit haruslah bertipe boolean, bernilai True atau False.")
    
    print("Fungsi median_imputation: parameter telah divalidasi.")
    
    # Copy data
    data = data.copy()
    subset_data = deepcopy(subset_data)
    
    if fit:
        imputation_data = {}
        for subset in subset_data:
            median_value = data[subset].median()
            imputation_data[subset] = median_value
        
        print(f"Fungsi median_imputation: proses fitting telah selesai, berikut hasilnya {imputation_data}.")
        return imputation_data
    
    else:
        print("Fungsi median_imputation: informasi count na sebelum dilakukan imputasi:")
        print(data.isna().sum())
        print("")

        data.fillna(subset_data, inplace=True)

        print("Fungsi median_imputation: informasi count na setelah dilakukan imputasi:")
        print(data.isna().sum())
        print("")

    return data

In [52]:
# Menentukan subset_data
subset_data = ['person_emp_length', 'loan_int_rate']

# Melakukan fitting untuk kalkulasi nilai median
subset_data = median_imputation(X_train, subset_data, True)


Fungsi median_imputation: parameter telah divalidasi.
Fungsi median_imputation: proses fitting telah selesai, berikut hasilnya {'person_emp_length': 4.0, 'loan_int_rate': 10.99}.


In [53]:
# Melakukan imputasi pada X_train
X_train = median_imputation(X_train, subset_data, False)



Fungsi median_imputation: parameter telah divalidasi.
Fungsi median_imputation: informasi count na sebelum dilakukan imputasi:
person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              734
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 2491
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64

Fungsi median_imputation: informasi count na setelah dilakukan imputasi:
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
dtype: int64



In [54]:
# Melakukan imputasi pada X_test
X_test = median_imputation(X_test, subset_data, False)


Fungsi median_imputation: parameter telah divalidasi.
Fungsi median_imputation: informasi count na sebelum dilakukan imputasi:
person_age                      0
person_income                   0
person_home_ownership           0
person_emp_length              77
loan_intent                     0
loan_grade                      0
loan_amnt                       0
loan_int_rate                 303
loan_percent_income             0
cb_person_default_on_file       0
cb_person_cred_hist_length      0
dtype: int64

Fungsi median_imputation: informasi count na setelah dilakukan imputasi:
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
dtype: int64



In [55]:
# Melakukan imputasi pada X_valid
X_valid = median_imputation(X_valid, subset_data, False)

Fungsi median_imputation: parameter telah divalidasi.
Fungsi median_imputation: informasi count na sebelum dilakukan imputasi:
person_age                      0
person_income                   0
person_home_ownership           0
person_emp_length              80
loan_intent                     0
loan_grade                      0
loan_amnt                       0
loan_int_rate                 312
loan_percent_income             0
cb_person_default_on_file       0
cb_person_cred_hist_length      0
dtype: int64

Fungsi median_imputation: informasi count na setelah dilakukan imputasi:
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
dtype: int64



### Categorical Data Encoding

In [56]:
def create_onehot_encoder(categories, path):
    """
    Fungsi untuk membuat one hot encoder berdasarkan kategori yang diberikan dan menyimpannya ke disk.

    Parameters       :
    categories (list): Daftar kategori yang akan dibuat encodernya.
    path       (str) : Lokasi pada disk di mana encoder akan disimpan.

    Returns          :
    OneHotEncoder    : Instance OneHotEncoder yang telah di-fitting.
    """
    
    # Validasi parameter
    if not isinstance(categories, list):
        raise RuntimeError("Fungsi create_onehot_encoder: parameter categories haruslah bertipe list, "
                           "berisi kategori yang akan dibuat encodernya.")
    
    if not isinstance(path, str):
        raise RuntimeError("Fungsi create_onehot_encoder: parameter path haruslah bertipe string, berisi " 
                           "lokasi pada disk komputer di mana encoder akan disimpan.")
    
    # Membuat instance OneHotEncoder
    ohe = OneHotEncoder()

    # Mengubah parameter categories menjadi numpy array 2 dimensi
    categories_array = np.array(categories).reshape(-1, 1)
    
    # Melakukan fitting encoder
    ohe.fit(categories_array)
    
    # Menyimpan encoder ke disk
    utils.serialize_data(ohe, path)
    
    # Menampilkan kategori yang telah dipelajari
    learned_categories = ohe.categories_[0].tolist()
    print(f"Kategori yang telah dipelajari adalah {learned_categories}")

    return ohe

In [57]:
# Definisikan konstanta kategori untuk kolom masing-masing
person_home_ownership = ["RENT", "OWN", "MORTGAGE", "OTHER"]
loan_intent           = ["PERSONAL", "EDUCATION", "MEDICAL", "VENTURE", "HOMEIMPROVEMENT", "DEBTCONSOLIDATION"]
loan_grade            = ["A", "B", "C", "D", "E", "F", "G"]
cb_person_default_on_file = ["Y", "N"]

# Buat dan simpan encoder untuk person_home_ownership
ohe_home_ownership = create_onehot_encoder(person_home_ownership, "models/ohe_home_ownership.pkl")

# Buat dan simpan encoder untuk loan_intent
ohe_loan_intent = create_onehot_encoder(loan_intent, "models/ohe_loan_intent.pkl")

# Buat dan simpan encoder untuk loan_grade
ohe_loan_grade = create_onehot_encoder(loan_grade, "models/ohe_loan_grade.pkl")

# Buat dan simpan encoder untuk cb_person_default_on_file
ohe_default_on_file = create_onehot_encoder(cb_person_default_on_file, "models/ohe_default_on_file.pkl")


Kategori yang telah dipelajari adalah ['MORTGAGE', 'OTHER', 'OWN', 'RENT']
Kategori yang telah dipelajari adalah ['DEBTCONSOLIDATION', 'EDUCATION', 'HOMEIMPROVEMENT', 'MEDICAL', 'PERSONAL', 'VENTURE']
Kategori yang telah dipelajari adalah ['A', 'B', 'C', 'D', 'E', 'F', 'G']
Kategori yang telah dipelajari adalah ['N', 'Y']


### Numerical Data Encoding

In [58]:
def ohe_transform(dataset, subset, prefix, ohe):
   
    """
    Melakukan One-Hot Encoding pada kolom kategorik dalam dataset.

    Parameter:
    -----------
    dataset: pd.DataFrame
             DataFrame yang berisi data yang ingin dilakukan pengkodean.
    subset : str
             Nama kolom dalam dataset yang berisi data kategorik yang akan di-encode.
    prefix : str
             Nama awalan yang akan disematkan pada kolom hasil pengkodean.
    ohe    : OneHotEncoder
             Encoder yang telah dilatih sebelumnya dengan kategori khusus.

    Returns:
    --------
    pd.DataFrame
        DataFrame yang telah dilakukan pengkodean pada kolom kategorik yang ditentukan.
    
    Raises:
    -------
    RuntimeError
        Jika parameter `dataset` bukan bertipe DataFrame.
        Jika parameter `subset` bukan bertipe str.
        Jika parameter `prefix` bukan bertipe str.
        Jika parameter `ohe` bukan bertipe OneHotEncoder.
        Jika kolom yang ditentukan oleh parameter `subset` tidak ditemukan dalam dataset.
    """
    
    # Validasi parameter dataset
    if not isinstance(dataset, pd.DataFrame):
        raise RuntimeError("Fungsi ohe_transform: parameter dataset harus bertipe DataFrame!")

    # Validasi parameter ohe
    if not isinstance(ohe, OneHotEncoder):
        raise RuntimeError("Fungsi ohe_transform: parameter ohe harus bertipe OneHotEncoder!")

    # Validasi parameter prefix
    if not isinstance(prefix, str):
        raise RuntimeError("Fungsi ohe_transform: parameter prefix harus bertipe str!")

    # Validasi parameter subset
    if not isinstance(subset, str):
        raise RuntimeError("Fungsi ohe_transform: parameter subset harus bertipe str!")

    # Validasi keberadaan subset dalam dataset
    try:
        dataset.columns.tolist().index(subset)
    except ValueError:
        raise RuntimeError("Fungsi ohe_transform: parameter subset string tidak ditemukan dalam daftar kolom yang terdapat pada parameter dataset.")

    print("Fungsi ohe_transform: parameter telah divalidasi.")
    
    # Membuat duplikat dataset
    dataset = dataset.copy()

    # Menampilkan daftar nama kolom sebelum pengkodean
    print(f"Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah {list(dataset.columns)}.")
    
    # Membuat nama kolom untuk hasil pengkodean
    col_names = [f"{prefix}_{col_name}" for col_name in ohe.categories_[0].tolist()]
    
    # Melakukan pengkodean
    encoded = pd.DataFrame(
        ohe.transform(dataset[[subset]]).toarray(),
        columns=col_names,
        index=dataset.index
    )

    # Menggabungkan hasil pengkodean dengan dataset
    dataset = pd.concat([dataset, encoded], axis=1)
    
    # Menghapus kolom asli yang sudah dikodekan
    dataset.drop(columns=[subset], inplace=True)
    
    # Menampilkan daftar nama kolom setelah pengkodean
    print(f"Fungsi ohe_transform: daftar nama kolom setelah dilakukan pengkodean adalah {list(dataset.columns)}.")
    print()
    
    return dataset


In [59]:
# Melakukan encoding data kategorik untuk X_train
X_train = ohe_transform(X_train, "person_home_ownership", "home_ownership", ohe_home_ownership)
X_train = ohe_transform(X_train, "loan_intent", "loan_intent", ohe_loan_intent)
X_train = ohe_transform(X_train, "loan_grade", "loan_grade", ohe_loan_grade)
X_train = ohe_transform(X_train, "cb_person_default_on_file", "default_on_file", ohe_default_on_file)



Fungsi ohe_transform: parameter telah divalidasi.
Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah ['person_age', 'person_income', 'person_home_ownership', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length'].
Fungsi ohe_transform: daftar nama kolom setelah dilakukan pengkodean adalah ['person_age', 'person_income', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length', 'home_ownership_MORTGAGE', 'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT'].

Fungsi ohe_transform: parameter telah divalidasi.
Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah ['person_age', 'person_income', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_fil

C:\Users\nggar\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
C:\Users\nggar\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
C:\Users\nggar\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
C:\Users\nggar\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


In [60]:
# Melakukan encoding data kategorik untuk X_test
X_test = ohe_transform(X_test, "person_home_ownership", "home_ownership", ohe_home_ownership)
X_test = ohe_transform(X_test, "loan_intent", "loan_intent", ohe_loan_intent)
X_test = ohe_transform(X_test, "loan_grade", "loan_grade", ohe_loan_grade)
X_test = ohe_transform(X_test, "cb_person_default_on_file", "default_on_file", ohe_default_on_file)



Fungsi ohe_transform: parameter telah divalidasi.
Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah ['person_age', 'person_income', 'person_home_ownership', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length'].
Fungsi ohe_transform: daftar nama kolom setelah dilakukan pengkodean adalah ['person_age', 'person_income', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length', 'home_ownership_MORTGAGE', 'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT'].

Fungsi ohe_transform: parameter telah divalidasi.
Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah ['person_age', 'person_income', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_fil

C:\Users\nggar\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
C:\Users\nggar\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
C:\Users\nggar\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
C:\Users\nggar\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


In [61]:
# Melakukan encoding data kategorik untuk X_valid
X_valid = ohe_transform(X_valid, "person_home_ownership", "home_ownership", ohe_home_ownership)
X_valid = ohe_transform(X_valid, "loan_intent", "loan_intent", ohe_loan_intent)
X_valid = ohe_transform(X_valid, "loan_grade", "loan_grade", ohe_loan_grade)
X_valid = ohe_transform(X_valid, "cb_person_default_on_file", "default_on_file", ohe_default_on_file)


Fungsi ohe_transform: parameter telah divalidasi.
Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah ['person_age', 'person_income', 'person_home_ownership', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length'].
Fungsi ohe_transform: daftar nama kolom setelah dilakukan pengkodean adalah ['person_age', 'person_income', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length', 'home_ownership_MORTGAGE', 'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT'].

Fungsi ohe_transform: parameter telah divalidasi.
Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah ['person_age', 'person_income', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_fil

C:\Users\nggar\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
C:\Users\nggar\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
C:\Users\nggar\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
C:\Users\nggar\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


In [62]:
X_train

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,home_ownership_MORTGAGE,home_ownership_OTHER,home_ownership_OWN,...,loan_intent_VENTURE,loan_grade_A,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G,default_on_file_N,default_on_file_Y
15884,25,241875,4.0,16000,7.05,0.07,4,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
15138,21,18000,5.0,1500,12.18,0.08,4,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7474,25,53000,10.0,16000,12.53,0.30,2,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
18212,28,16800,4.0,5000,13.98,0.30,8,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6493,25,50000,2.0,10000,7.90,0.20,2,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14621,25,98000,9.0,25000,16.45,0.26,4,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
18736,30,65000,0.0,16000,13.06,0.25,8,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1663,22,20000,7.0,1675,7.74,0.08,3,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
18257,29,110000,4.0,24000,18.39,0.22,8,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [63]:
# Serialize data
utils.serialize_data(X_train, 'data/processed/X_train_prep.pkl')


['data/processed/X_train_prep.pkl']

In [64]:
utils.serialize_data(X_test, 'data/processed/X_test_prep.pkl')


['data/processed/X_test_prep.pkl']

In [65]:
utils.serialize_data(X_valid, 'data/processed/X_valid_prep.pkl')


['data/processed/X_valid_prep.pkl']

In [66]:
utils.serialize_data(y_train, 'data/processed/y_train_prep.pkl')

['data/processed/y_train_prep.pkl']

In [67]:
utils.serialize_data(y_test, 'data/processed/y_test_prep.pkl')

['data/processed/y_test_prep.pkl']

In [68]:
utils.serialize_data(y_valid, 'data/processed/y_valid_prep.pkl')

['data/processed/y_valid_prep.pkl']